## Customer Reviews Enrichment


In [27]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

nltk.download('vader_lexicon')

def fetch_data_from_sql():

# Connect to the SQL Server database
    conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-D8BRM8F;'  
                      'Database=MarketingAnalytics;'
                      'Trusted_Connection=yes;')

    query = "SELECT ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText FROM customer_reviews"

    df = pd.read_sql(query, conn)
    conn.close()
    return df

customer_reviews_df = fetch_data_from_sql()
print(customer_reviews_df)

sia = SentimentIntensityAnalyzer()

def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']

def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating >= 4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score < -0.05:
        if rating <= 2:
            return 'Negative'
        elif rating == 3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
    else:
        if rating >= 4:
            return 'Positive'
        elif rating <= 2:
            return 'Negative'
        else:
            return 'Neutral'

#Define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.05:
        return '0.5 to 1.0'
    elif 0.0 <= score < 0.05:
        return '0.0 to 0.49'
    elif -0.05 <= score < 0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'

customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)

customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)
print(customer_reviews_df.head())



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
C:\Users\HP\AppData\Local\Temp\ipykernel_18184\1967761978.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


      ReviewID  CustomerID  ProductID  ReviewDate  Rating  \
0            1          77         18  2023-12-23       3   
1            2          80         19  2024-12-25       5   
2            3          50         13  2025-01-26       4   
3            4          78         15  2025-04-21       3   
4            5          64          2  2023-07-16       3   
...        ...         ...        ...         ...     ...   
1358      1359          28          4  2023-05-25       3   
1359      1360          58         12  2023-11-13       2   
1360      1361          96         15  2023-03-07       5   
1361      1362          99          2  2025-12-03       1   
1362      1363          16          4  2024-07-16       2   

                                      ReviewText  
0        Average  experience,  nothing  special.  
1                 The  quality  is    top-notch.  
2        Five  stars  for  the  quick  delivery.  
3       Good  quality,  but  could  be  cheaper.  
4        Ave